# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [52]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [53]:
csv_path = "../WeatherPy/weather_df.csv"
weather_df = pd.read_csv(csv_path, encoding="utf-8")
weather_df


,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Temperature,Wind Speed (mph)
0,Portland,90,US,1579838530,87,45.52,-122.68,54.00,11.41
1,Bredasdorp,81,ZA,1579838709,88,-34.53,20.04,66.00,4.70
2,Mataura,100,NZ,1579838711,42,-46.19,168.86,83.55,3.49
3,Isangel,75,VU,1579838712,58,-19.55,169.27,87.80,17.22
4,Fort Nelson,90,CA,1579838169,92,58.81,-122.70,14.00,3.36
...,...,...,...,...,...,...,...,...,...
570,Hanna,93,CA,1579839664,91,51.63,-111.90,19.99,4.00
571,Burriana,90,ES,1579839666,100,39.89,-0.09,50.00,2.24
572,Champerico,9,GT,1579839368,82,14.30,-91.92,74.01,5.10
573,San Andres,40,CO,1579839669,69,12.58,-81.70,78.80,6.93


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [54]:
gmaps.configure(api_key=g_key)


In [55]:
locations = weather_df[["Latitude", "Longitude"]].astype(float)

humidity = weather_df["Humidity"].astype(float)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [56]:
zero_clouds = weather_df.loc[weather_df['Cloudiness'] == 0]
low_wind = zero_clouds.loc[zero_clouds['Wind Speed (mph)'] <10]
min_temp = low_wind.loc[low_wind['Temperature']>=70]
ideal_weather = min_temp.loc[min_temp['Temperature']<=80]
ideal_weather

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Temperature,Wind Speed (mph)
45,Praia,0,CV,1579838781,56,14.92,-23.51,71.60,8.05
102,Vila do Maio,0,CV,1579838874,56,15.13,-23.22,71.60,8.05
161,Tabuk,0,PH,1579838988,69,17.42,121.44,78.06,2.26
168,Cockburn Town,0,TC,1579838999,59,21.46,-71.14,74.59,8.52
201,Sao Filipe,0,CV,1579838985,79,14.90,-24.50,70.84,6.55
326,Departamento de Maldonado,0,UY,1579838966,68,-34.67,-54.92,71.60,5.82
343,Cabo Rojo,0,PR,1579839288,77,18.09,-67.15,75.00,4.09
436,Santa Isabel do Rio Negro,0,BR,1579839294,74,-0.41,-65.02,79.20,2.19
493,Kanigiri,0,IN,1579839538,71,15.40,79.52,72.46,1.21
522,General Roca,0,AR,1579839585,29,-39.03,-67.58,77.00,4.70


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [57]:
ideal_weather.loc[:, ('Hotel Name')] = ""

params = {
    "radius": 50000,
    "keyword": "hotel",
    "key": g_key
}

for index, row in ideal_weather.iterrows():
  
    lat = row["Latitude"]
    lng = row["Longitude"]

    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    name_address = requests.get(base_url, params=params)

    name_address = name_address.json()

    try:
        ideal_weather.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        



In [58]:
ideal_weather

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Temperature,Wind Speed (mph),Hotel Name
45,Praia,0,CV,1579838781,56,14.92,-23.51,71.60,8.05,Hotel VIP Praia
102,Vila do Maio,0,CV,1579838874,56,15.13,-23.22,71.60,8.05,Hotel VIP Praia
161,Tabuk,0,PH,1579838988,69,17.42,121.44,78.06,2.26,Dreamwave Hotel
168,Cockburn Town,0,TC,1579838999,59,21.46,-71.14,74.59,8.52,East Bay Resort
201,Sao Filipe,0,CV,1579838985,79,14.90,-24.50,70.84,6.55,Hotel Xaguate
326,Departamento de Maldonado,0,UY,1579838966,68,-34.67,-54.92,71.60,5.82,Casapueblo Hotel
343,Cabo Rojo,0,PR,1579839288,77,18.09,-67.15,75.00,4.09,Rincon Beach Resort
436,Santa Isabel do Rio Negro,0,BR,1579839294,74,-0.41,-65.02,79.20,2.19,Hotel GSA
493,Kanigiri,0,IN,1579839538,71,15.40,79.52,72.46,1.21,Kaveri Inn
522,General Roca,0,AR,1579839585,29,-39.03,-67.58,77.00,4.70,Howard Johnson Neuquen


In [63]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal_weather.iterrows()]
locations = ideal_weather[["Latitude", "Longitude"]]


In [77]:
# Add marker layer ontop of heat map
humidity = ideal_weather["Humidity"].astype(float)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)


heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig




Figure(layout=FigureLayout(height='420px'))